In [12]:
import re
import time
import datetime
from crawling import *

In [13]:
def geturl(parsed_html):
    result = []
    elements = parsed_html.select("tbody > tr > td:nth-child(1) > a:nth-child(1)")
    for element in elements:
        result.append("https://namu.wiki" + element.attrs.get("href"))
    return result

def gettime(parsed_html):
    alltimes = parsed_html.select("tr time")
    lasttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[0].text)))
    firsttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[-1].text)))
    return (lasttime - firsttime).seconds//2

In [14]:
page_urls = {}
origin_url = r"https://namu.wiki/RecentChanges"
# get urls until input limit
url_limit = int(input())
while True:
    namu_recent = html_parser(origin_url, 5)
    if namu_recent is not None:
        # get url
        for url in geturl(namu_recent):
            page_urls[url] = page_urls.get(url, 0) + 1
        print("url number:", len(page_urls))
        if len(page_urls) > url_limit:
            break
        # get time dif
        timedif = gettime(namu_recent)
        print(f"waiting time: {timedif}s")
        time.sleep(timedif)

url number: 89


In [15]:
cnt = 0
titles,  categories ,paragraphs = [],[],[]
for target_url in page_urls:
    if cnt >= 5:
        break
    cnt += 1
    namu_keyword = html_parser(target_url)
    if namu_keyword is not None:
        article = namu_keyword.select_one("article")
        title = get_text(article.select_one("h1"))
        raw_categories = list(article.select_one("h2").next_siblings)[-1].select("li")
        category = "&".join(get_text(raw_categories))
        # remove unnecessary info
        for i in article.select("table"):         
            i.clear()
        # get paragraph data
        if article.select_one(".wiki-heading-content") is None:
            raw_paragraphs = article.select(".wiki-paragraph")
        else:
            raw_paragraphs = article.select(".wiki-heading-content>.wiki-paragraph")
        paragraph = "&".join(get_text(raw_paragraphs))
        titles.append(title)
        categories.append(category)
        paragraphs.append(paragraph)

In [16]:
import pandas
df = pandas.DataFrame(data={"title":titles, "category":categories, "paragraph":paragraphs})
df

,title,category,paragraph
0,파일:전독시 표지_웹소설 단행본_1부 3권_대만.png,파일/전지적 독자 시점,&이미지의 자세한 설명을 적어 주세요.
1,신(엘피스 전기),엘피스 전기/설정,엘피스 전기 시리즈에 나오는 설정이며 신이 되기 위해서는 마나레벨을 100레벨을 찍...
2,Midnight Guest,프로미스나인/음반 목록&2022년 음반,2022년 1월 17일 발매된 프로미스나인의 4번째 미니 앨범이다.&&&&& 은...
3,자린고비,한자어,"한국의 대표적인 구두쇠, 혹은 구두쇠의 대명사.&자린고비라는 말의 유래에 대해서는 ..."
4,틀:OTT 서비스,둘러보기 틀/방송&VOD,&
